In [1]:
def merge_bin(bin_a, bin_b, ori_buckets):

    '''合并分箱，返回新的边界点'''
    new_bin = pd.Interval(left=bin_a.left, right=bin_b.right,closed='right')
    #将原先的分箱删除并生成新合并的分箱
    new_buckets = ori_buckets.remove_categories([bin_a, bin_b]).dropna()
    new_buckets = new_buckets.add_categories([new_bin])
    new_buckets = pd.CategoricalIndex(new_buckets.categories).sort_values()

    new_cut = np.unique(np.array([[i.left, i.right] for i in new_buckets]).flatten()).tolist()

    #返回合并后新的边界点重新分箱以及合并后的分箱数用于终止合并
    return new_cut, len(new_buckets)

def chi_bins():
    data = pd.read_csv("./germancredit.csv")
    data['creditability'] = data['creditability'].apply(lambda x: 1 if x == 'bad' else 0)
    bins = 5
    init_bins = data['duration.in.month'].unique()
    cut_points = sorted(init_bins)
    #由于pandas.cut分箱区间是左开右闭，所以将切分点的最小值和最大值都减小或增加一个值以使区间包含最大值和最小值，
    #避免某些样本无法被分箱
    cut_points[0], cut_points[-1] = cut_points[0]-0.1, cut_points[-1]+0.1
    while True:
        #cut_points = [cut_points[i] for i in range(1, len(cut_points) - 1, 2)]
        df = pd.DataFrame(data=pd.cut(x=data['duration.in.month'], bins=cut_points)).reset_index() \
            .rename(columns={'index': 'sample', 'duration.in.month': 'bins'})
        df['target'] = data['creditability'].astype(int)
        buckets = df['bins'].value_counts().index.sort_values()
        chi_values = []
        for i in range(len(buckets)-1):

            #print(buckets[i],buckets[i+1])
            bin_a = df[df['bins']==buckets[i]]
            bin_b = df[df['bins']==buckets[i+1]]


            #获取两个bin中的分布，即好人数，坏人数以及bin中的样本数
            bin_a_dist = [bin_a['target'].sum(), bin_a['target'].count()-bin_a['target'].sum()
                         , bin_a['target'].count()]
            bin_b_dist = [bin_b['target'].sum(), bin_b['target'].count()-bin_b['target'].sum()
                , bin_b['target'].count()]
            #合并后的分布
            d = np.array([bin_a_dist,bin_b_dist])

            chi_values.append(chi2_contingency(d)[0])

        #获取最小卡方值的索引，相邻两个分箱合并
        min_chi_index = np.argmin(chi_values)
        bin_a = buckets[min_chi_index]
        bin_b = buckets[min_chi_index + 1]
        cut_points, b_num = merge_bin(bin_a, bin_b, buckets)

        if b_num==bins:
            break


    bad_rate = (df.groupby('bins').agg({'target': 'sum'}) / df.groupby('bins').agg({'target': 'count'})) \
        .reset_index().rename(columns={'target': 'bad_rate'})
    df_counts = pd.DataFrame(data=df['bins'].value_counts()).reset_index() \
        .rename(columns={'index': 'bins', 'bins': 'sample_num'})

In [2]:
def cal_chi(freq_array):
    """ 计算卡方值
    Args:
        freq_array: Array，待计算卡方值的二维数组，频数统计结果
    Returns:
        卡方值，float
    """
    # 检查是否为二维数组
    assert(freq_array.ndim==2)
    
    # 计算每列的频数之和
    col_nums = freq_array.sum(axis=0)
    # 计算每行的频数之和
    row_nums = freq_array.sum(axis=1)
    # 计算总频数
    nums = freq_array.sum()
    # 计算期望频数
    E_nums = np.ones(freq_array.shape) * col_nums / nums
    E_nums = (E_nums.T * row_nums).T
    # 计算卡方值
    tmp_v = (freq_array - E_nums)**2 / E_nums
    # 如果期望频数为0，则计算结果记为0
    tmp_v[E_nums==0] = 0
    chi_v = tmp_v.sum()
    return chi_v